In [ ]:
!pip install split-folders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# display, transform, read, split ...
import numpy as np
import cv2 as cv
import os
import splitfolders
import matplotlib.pyplot as plt

# tensorflow
import tensorflow.keras as keras
import tensorflow as tf

# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
main_folder="drive/MyDrive/dataset-resized"

In [ ]:
print(os.listdir(main_folder))

['glass', 'plastic', 'trash', 'paper', 'metal', 'cardboard']


In [ ]:
splitfolders.ratio(main_folder, output="dataset-split", seed=1337, ratio=(0.7, 0.2, 0.1), group_prefix=None, move=False)

Copying files: 2537 files [00:42, 59.15 files/s] 


In [ ]:
new_folder = "dataset-split"

In [ ]:
print(os.listdir(new_folder))

['test', 'train', 'val']


In [ ]:
print(os.listdir(new_folder+"/train"))

['cardboard', 'plastic', 'metal', 'glass', 'paper', 'trash']


In [ ]:
train_datagen = ImageDataGenerator(
                rotation_range=25,
                width_shift_range=0.2,
                height_shift_range=0.2,
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# define classes name
class_names = ['trash','plastic','paper','metal','glass','cardboard']

In [ ]:
# training data
train_generator = train_datagen.flow_from_directory(
    directory="dataset-split/train",
    classes = class_names,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
)

Found 1773 images belonging to 6 classes.


In [ ]:
# validation data
valid_generator = val_datagen.flow_from_directory(
    directory="dataset-split/val/",
    classes = class_names,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
)

Found 505 images belonging to 6 classes.


In [ ]:
# test data
test_generator = val_datagen.flow_from_directory(
    directory="dataset-split/test/",
    classes = class_names,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
)

Found 259 images belonging to 6 classes.


In [ ]:
# ResNet50 model
resnet_50 = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in resnet_50.layers:
    layer.trainable = False

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
# build the entire model
x = resnet_50.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(6, activation='softmax')(x)
model = Model(inputs = resnet_50.input, outputs = predictions)

In [ ]:
# define training function
def trainModel(model, epochs, optimizer):
    batch_size = 32
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model.fit(train_generator, validation_data=valid_generator, epochs=epochs, batch_size=batch_size)

In [ ]:
# launch the training
model_history = trainModel(model = model, epochs = 50, optimizer = "Adam")

Epoch 1/50
56/56 [==============================] - 43s 558ms/step - loss: 1.8960 - accuracy: 0.2138 - val_loss: 1.7725 - val_accuracy: 0.2020
Epoch 2/50
56/56 [==============================] - 28s 496ms/step - loss: 1.7731 - accuracy: 0.1997 - val_loss: 1.7484 - val_accuracy: 0.2337
Epoch 3/50
56/56 [==============================] - 26s 449ms/step - loss: 1.7658 - accuracy: 0.2171 - val_loss: 1.7480 - val_accuracy: 0.2337
Epoch 4/50
56/56 [==============================] - 26s 458ms/step - loss: 1.7477 - accuracy: 0.1980 - val_loss: 1.7498 - val_accuracy: 0.2020
Epoch 5/50
56/56 [==============================] - 26s 466ms/step - loss: 1.7450 - accuracy: 0.2036 - val_loss: 1.7403 - val_accuracy: 0.2337
Epoch 6/50
56/56 [==============================] - 26s 463ms/step - loss: 1.7359 - accuracy: 0.2245 - val_loss: 1.7341 - val_accuracy: 0.2337
Epoch 7/50
56/56 [==============================] - 27s 491ms/step - loss: 1.7310 - accuracy: 0.2217 - val_loss: 1.7335 - val_accuracy: 0.2337

In [ ]:
x = resnet_50.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(6, activation='softmax')(x)
model = Model(inputs = resnet_50.input, outputs = predictions)

In [ ]:
# launch the training
model_history = trainModel(model = model, epochs = 50, optimizer = "Adam")

Epoch 1/50
56/56 [==============================] - 31s 472ms/step - loss: 1.6515 - accuracy: 0.2848 - val_loss: 1.6491 - val_accuracy: 0.3010
Epoch 2/50
56/56 [==============================] - 26s 462ms/step - loss: 1.6573 - accuracy: 0.2843 - val_loss: 1.6476 - val_accuracy: 0.3168
Epoch 3/50
56/56 [==============================] - 27s 486ms/step - loss: 1.6506 - accuracy: 0.2854 - val_loss: 1.6535 - val_accuracy: 0.3149
Epoch 4/50
56/56 [==============================] - 26s 461ms/step - loss: 1.6418 - accuracy: 0.2876 - val_loss: 1.6525 - val_accuracy: 0.3228
Epoch 5/50
56/56 [==============================] - 26s 467ms/step - loss: 1.6474 - accuracy: 0.2786 - val_loss: 1.6416 - val_accuracy: 0.3248
Epoch 6/50
56/56 [==============================] - 26s 462ms/step - loss: 1.6485 - accuracy: 0.2831 - val_loss: 1.6441 - val_accuracy: 0.3287
Epoch 7/50
56/56 [==============================] - 26s 463ms/step - loss: 1.6521 - accuracy: 0.2961 - val_loss: 1.6414 - val_accuracy: 0.3248

In [ ]:
score = model.evaluate(test_generator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.6317155361175537
Test accuracy: 0.3281853199005127


In [ ]:
# training data
train_generator = train_datagen.flow_from_directory(
    directory="dataset-split/train",
    classes = class_names,
    target_size=(150, 150),
    batch_size=32,
    class_mode="sparse",
)

Found 1773 images belonging to 6 classes.


In [ ]:
# validation data
valid_generator = val_datagen.flow_from_directory(
    directory="dataset-split/val/",
    classes = class_names,
    target_size=(150, 150),
    batch_size=32,
    class_mode="sparse",
)

Found 505 images belonging to 6 classes.


In [ ]:
# test data
test_generator = val_datagen.flow_from_directory(
    directory="dataset-split/test/",
    classes = class_names,
    target_size=(150, 150),
    batch_size=32,
    class_mode="sparse",
)

Found 259 images belonging to 6 classes.


In [ ]:
# launch the training
model_history = trainModel(model = model, epochs = 50, optimizer = "Adam")

Epoch 1/50
56/56 [==============================] - 23s 296ms/step - loss: 1.6919 - accuracy: 0.2290 - val_loss: 1.6473 - val_accuracy: 0.3030
Epoch 2/50
56/56 [==============================] - 15s 272ms/step - loss: 1.6746 - accuracy: 0.2792 - val_loss: 1.6495 - val_accuracy: 0.2990
Epoch 3/50
56/56 [==============================] - 14s 249ms/step - loss: 1.6521 - accuracy: 0.2854 - val_loss: 1.6312 - val_accuracy: 0.3089
Epoch 4/50
56/56 [==============================] - 14s 251ms/step - loss: 1.6544 - accuracy: 0.2820 - val_loss: 1.6472 - val_accuracy: 0.2970
Epoch 5/50
56/56 [==============================] - 14s 249ms/step - loss: 1.6506 - accuracy: 0.2916 - val_loss: 1.6327 - val_accuracy: 0.3228
Epoch 6/50
56/56 [==============================] - 14s 251ms/step - loss: 1.6658 - accuracy: 0.2707 - val_loss: 1.6510 - val_accuracy: 0.2891
Epoch 7/50
56/56 [==============================] - 14s 255ms/step - loss: 1.6630 - accuracy: 0.2798 - val_loss: 1.6696 - val_accuracy: 0.2871

In [ ]:
score = model.evaluate(test_generator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.6513731479644775
Test accuracy: 0.3281853199005127
